In [3]:
import numpy as np
import pandas as pd
import random as rd
from liblinear.liblinearutil import *
from sklearn.model_selection import train_test_split


In [4]:
demographics = pd.read_csv('demographics.csv')
location = pd.read_csv('location.csv')
satisfaction = pd.read_csv('satisfaction.csv')
services = pd.read_csv('services.csv')
status = pd.read_csv('status.csv')
Train_ID = pd.read_csv('Train_IDs.csv')
Test_ID = pd.read_csv('Test_IDs.csv')

demographics



,Customer ID,Count,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents
0,1746-TGTWV,1.0,Male,78.0,No,Yes,No,No,0.0
1,8499-BRXTD,1.0,Female,74.0,No,Yes,Yes,NaN,1.0
2,2041-JIJCI,1.0,Male,71.0,No,Yes,No,Yes,3.0
3,5832-EXGTT,1.0,Female,78.0,No,Yes,Yes,Yes,1.0
4,8350-NYMVI,NaN,NaN,80.0,No,Yes,NaN,Yes,1.0
...,...,...,...,...,...,...,...,...,...
6158,7609-NRNCA,NaN,Female,NaN,No,No,No,No,0.0
6159,4835-YSJMR,1.0,Male,38.0,No,No,NaN,Yes,2.0
6160,5067-DGXLL,1.0,NaN,30.0,No,No,Yes,Yes,2.0
6161,3129-AAQOU,1.0,Female,32.0,No,No,Yes,Yes,2.0


In [5]:
print(demographics.isnull().sum())
print(pd.DataFrame(demographics).info())

Customer ID               0
Count                   770
Gender                  770
Age                     770
Under 30                770
Senior Citizen          770
Married                 770
Dependents              770
Number of Dependents    770
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6163 entries, 0 to 6162
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           6163 non-null   object 
 1   Count                 5393 non-null   float64
 2   Gender                5393 non-null   object 
 3   Age                   5393 non-null   float64
 4   Under 30              5393 non-null   object 
 5   Senior Citizen        5393 non-null   object 
 6   Married               5393 non-null   object 
 7   Dependents            5393 non-null   object 
 8   Number of Dependents  5393 non-null   float64
dtypes: float64(3), object(6)
memory usage: 433.5+ KB
None


In [6]:
x = pd.DataFrame(demographics)
x = x.drop(['Count'],axis = 1)
x['Age'] = pd.to_numeric(x.Age,errors='coerce')


x.rename(columns={'Number of Dependents':'Number_of_Dependents'},inplace=True)
x['Number_of_Dependents'] = pd.to_numeric(x.Number_of_Dependents,errors='coerce')
x.rename(columns={'Senior Citizen':'Senior_Citizen'},inplace=True)
x.rename(columns={'Under 30':'Under_30'},inplace=True)
#x.rename(columns={'Customer ID':'Customer_ID'},inplace=True)


x['Gender']=x['Gender'].replace(['Male','Female'],[1,0])
x['Under_30']=x['Under_30'].replace(['Yes','No'],[1,0])
x['Senior_Citizen']=x['Senior_Citizen'].replace(['Yes','No'],[1,0])
x['Married']=x['Married'].replace(['Yes','No'],[1,0])
x['Dependents']=x['Dependents'].replace(['Yes','No'],[1,0])

#next : fill nan
x.Gender = x.Gender.fillna(method='pad')
x.Age = x.Age.fillna(x.Age.mean())
x.Under_30 = x.Under_30.fillna(method='pad')
x.Senior_Citizen = x.Senior_Citizen.fillna(method='pad')
x.Married = x.Married.fillna(method='pad')
x.Dependents = x.Dependents.fillna(method='pad')
x.Number_of_Dependents = x.Number_of_Dependents.fillna(method='pad')


x.Gender = x.Gender.astype(int)
x.Age = x.Age.astype(int)
x.Under_30 = x.Under_30.astype(int)
x.Senior_Citizen = x.Gender.astype(int)
x.Married = x.Married.astype(int)
x.Dependents = x.Dependents.astype(int)
x.Number_of_Dependents = x.Number_of_Dependents.astype(int)

print(x.isnull().sum())
print(x.info())

pd.DataFrame(x)


Customer ID             0
Gender                  0
Age                     0
Under_30                0
Senior_Citizen          0
Married                 0
Dependents              0
Number_of_Dependents    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6163 entries, 0 to 6162
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Customer ID           6163 non-null   object
 1   Gender                6163 non-null   int64 
 2   Age                   6163 non-null   int64 
 3   Under_30              6163 non-null   int64 
 4   Senior_Citizen        6163 non-null   int64 
 5   Married               6163 non-null   int64 
 6   Dependents            6163 non-null   int64 
 7   Number_of_Dependents  6163 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 385.3+ KB
None


,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents
0,1746-TGTWV,1,78,0,1,0,0,0
1,8499-BRXTD,0,74,0,0,1,0,1
2,2041-JIJCI,1,71,0,1,0,1,3
3,5832-EXGTT,0,78,0,0,1,1,1
4,8350-NYMVI,0,80,0,0,1,1,1
...,...,...,...,...,...,...,...,...
6158,7609-NRNCA,0,46,0,0,0,0,0
6159,4835-YSJMR,1,38,0,1,0,1,2
6160,5067-DGXLL,1,30,0,1,1,1,2
6161,3129-AAQOU,0,32,0,0,1,1,2


In [7]:
#location csv fillna with mean first
#merge demographics and location,keep only Latitude,Longitude in location

location.Latitude = location.Latitude.fillna(location.Latitude.mean())
location.Longitude = location.Longitude.fillna(location.Longitude.mean())
x = pd.merge(x,location,on='Customer ID',how='outer')
x=x.drop(['Count'],axis=1)
x=x.drop(['Country'],axis=1)
x=x.drop(['State'],axis=1)
x=x.drop(['City'],axis=1)
x=x.drop(['Zip Code'],axis=1)
x=x.drop(['Lat Long'],axis=1)


pd.DataFrame(x)

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,Latitude,Longitude
0,1746-TGTWV,1.0,78.0,0.0,1.0,0.0,0.0,0.0,34.023810,-118.156582
1,8499-BRXTD,0.0,74.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN
2,2041-JIJCI,1.0,71.0,0.0,1.0,0.0,1.0,3.0,34.108833,-118.229715
3,5832-EXGTT,0.0,78.0,0.0,0.0,1.0,1.0,1.0,33.936291,-118.332639
4,8350-NYMVI,0.0,80.0,0.0,0.0,1.0,1.0,1.0,36.192042,-118.020188
...,...,...,...,...,...,...,...,...,...,...
6940,5183-KLYEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.686740,-117.186612
6941,8086-OVPWV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.898613,-117.202937
6942,3677-IYRBF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.752938,-116.230055
6943,5583-SXDAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.680031,-116.171678


In [8]:
#satisfaction csv fillna with mean first

satisfaction['Satisfaction Score'] = satisfaction['Satisfaction Score'].fillna(satisfaction['Satisfaction Score'].mean())
satisfaction['Satisfaction Score'] = satisfaction['Satisfaction Score'].astype(int)
x = pd.merge(x,satisfaction,on='Customer ID',how='outer')

pd.DataFrame(x)

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,Latitude,Longitude,Satisfaction Score
0,1746-TGTWV,1.0,78.0,0.0,1.0,0.0,0.0,0.0,34.023810,-118.156582,3.0
1,8499-BRXTD,0.0,74.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,3.0
2,2041-JIJCI,1.0,71.0,0.0,1.0,0.0,1.0,3.0,34.108833,-118.229715,3.0
3,5832-EXGTT,0.0,78.0,0.0,0.0,1.0,1.0,1.0,33.936291,-118.332639,2.0
4,8350-NYMVI,0.0,80.0,0.0,0.0,1.0,1.0,1.0,36.192042,-118.020188,2.0
...,...,...,...,...,...,...,...,...,...,...,...
7026,9432-RUVSL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
7027,3486-KHMLI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
7028,7817-BOQPW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
7029,4767-HZZHQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0


In [9]:
#services drop the unessential column
services = services.drop(['Count'],axis=1)
services = services.drop(['Monthly Charge'],axis=1)
services = services.drop(['Total Charges'],axis=1)
services = services.drop(['Total Refunds'],axis=1)
services = services.drop(['Total Long Distance Charges'],axis=1)
services = services.drop(['Avg Monthly Long Distance Charges'],axis=1)


In [10]:
#fillna and type convertion
services['Quarter'] = services['Quarter'].replace(['Q1','Q2','Q3','Q4'],[1,2,3,4])
services['Quarter'] = services['Quarter'].fillna(method='pad')
services['Quarter'] = services['Quarter'].astype(int)

services['Referred a Friend'] = services['Referred a Friend'].replace(['Yes','No'],[1,0])
services['Referred a Friend'] = services['Referred a Friend'].fillna(method='pad')
services['Referred a Friend'] = services['Referred a Friend'].astype(int)

services['Number of Referrals'] = services['Number of Referrals'].fillna(method='pad')
services['Number of Referrals'] = services['Number of Referrals'].astype(int)

services['Tenure in Months'] = services['Tenure in Months'].fillna(services['Tenure in Months'].mean())
services['Tenure in Months'] = services['Tenure in Months'].astype(int)

services['Offer'] = services['Offer'].replace(['None','Offer A','Offer B','Offer C','Offer D','Offer E'],[0,1,2,3,4,5])
services['Offer'] = services['Offer'].fillna(method='pad')
services['Offer'] = services['Offer'].astype(int)

services['Phone Service'] = services['Phone Service'].replace(['Yes','No'],[1,0])
services['Phone Service'] = services['Phone Service'].fillna(method='pad')
services['Phone Service'] = services['Phone Service'].astype(int)

services['Multiple Lines'] = services['Multiple Lines'].replace(['Yes','No'],[1,0])
services['Multiple Lines'] = services['Multiple Lines'].fillna(method='pad')
services['Multiple Lines'] = services['Multiple Lines'].astype(int)

services['Internet Service'] = services['Internet Service'].replace(['Yes','No'],[1,0])
services['Internet Service'] = services['Internet Service'].fillna(method='pad')
services['Internet Service'] = services['Internet Service'].astype(int)

services['Internet Type'] = services['Internet Type'].replace(['None','DSL','Fiber Optic','Cable'],[0,1,2,3])
services['Internet Type'] = services['Internet Type'].fillna(method='pad')
services['Internet Type'] = services['Internet Type'].astype(int)

services['Avg Monthly GB Download'] = services['Avg Monthly GB Download'].fillna(services['Avg Monthly GB Download'].mean())
services['Avg Monthly GB Download'] = services['Avg Monthly GB Download'].astype(int)

services['Online Security'] = services['Online Security'].replace(['Yes','No'],[1,0])
services['Online Security'] = services['Online Security'].fillna(method='pad')
services['Online Security'] = services['Online Security'].astype(int)

services['Online Backup'] = services['Online Backup'].replace(['Yes','No'],[1,0])
services['Online Backup'] = services['Online Backup'].fillna(method='pad')
services['Online Backup'] = services['Online Backup'].astype(int)

services['Device Protection Plan'] = services['Device Protection Plan'].replace(['Yes','No'],[1,0])
services['Device Protection Plan'] = services['Device Protection Plan'].fillna(method='pad')
services['Device Protection Plan'] = services['Device Protection Plan'].fillna(method='pad')
services['Device Protection Plan'] = services['Device Protection Plan'].fillna(services['Device Protection Plan'].mean())
services['Device Protection Plan'] = services['Device Protection Plan'].astype(int)

services['Premium Tech Support'] = services['Premium Tech Support'].replace(['Yes','No'],[1,0])
services['Premium Tech Support'] = services['Premium Tech Support'].fillna(method='pad')
services['Premium Tech Support'] = services['Premium Tech Support'].astype(int)

services['Streaming TV'] = services['Streaming TV'].replace(['Yes','No'],[1,0])
services['Streaming TV'] = services['Streaming TV'].fillna(method='pad')
services['Streaming TV'] = services['Streaming TV'].astype(int)

services['Streaming Movies'] = services['Streaming Movies'].replace(['Yes','No'],[1,0])
services['Streaming Movies'] = services['Streaming Movies'].fillna(method='pad')
services['Streaming Movies'] = services['Streaming Movies'].astype(int)

services['Streaming Music'] = services['Streaming Music'].replace(['Yes','No'],[1,0])
services['Streaming Music'] = services['Streaming Music'].fillna(method='pad')
services['Streaming Music'] = services['Streaming Music'].astype(int)

services['Unlimited Data'] = services['Unlimited Data'].replace(['Yes','No'],[1,0])
services['Unlimited Data'] = services['Unlimited Data'].fillna(method='pad')
services['Unlimited Data'] = services['Unlimited Data'].astype(int)

services['Contract'] = services['Contract'].replace(['Month-to-Month', 'One Year', 'Two Year'],[0,1,2])
services['Contract'] = services['Contract'].fillna(method='pad')
services['Contract'] = services['Contract'].astype(int)

services['Paperless Billing'] = services['Paperless Billing'].replace(['Yes', 'No'],[1,0])
services['Paperless Billing'] = services['Paperless Billing'].fillna(method='pad')
services['Paperless Billing'] = services['Paperless Billing'].astype(int)

services['Payment Method'] = services['Payment Method'].replace(['Bank Withdrawal', 'Credit Card', 'Mailed Check'],[0,1,2])
services['Payment Method'] = services['Payment Method'].fillna(method='pad')
services['Payment Method'] = services['Payment Method'].astype(int)

services['Total Revenue'] = services['Total Revenue'].fillna(services['Total Revenue'].mean())


#merge services and x 
x = pd.merge(x,services,on='Customer ID',how='outer')

print(x.isnull().sum())
pd.DataFrame(x)

Customer ID                    0
Gender                       880
Age                          880
Under_30                     880
Senior_Citizen               880
Married                      880
Dependents                   880
Number_of_Dependents         880
Latitude                     880
Longitude                    880
Satisfaction Score           880
Quarter                      880
Referred a Friend            880
Number of Referrals          880
Tenure in Months             880
Offer                        880
Phone Service                880
Multiple Lines               880
Internet Service             880
Internet Type                880
Avg Monthly GB Download      880
Online Security              880
Online Backup                880
Device Protection Plan       880
Premium Tech Support         880
Streaming TV                 880
Streaming Movies             880
Streaming Music              880
Unlimited Data               880
Contract                     880
Paperless 

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,Latitude,Longitude,...,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Total Extra Data Charges,Total Revenue
0,1746-TGTWV,1.0,78.0,0.0,1.0,0.0,0.0,0.0,34.023810,-118.156582,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,20.0,59.650000
1,8499-BRXTD,0.0,74.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1024.100000
2,2041-JIJCI,1.0,71.0,0.0,1.0,0.0,1.0,3.0,34.108833,-118.229715,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5832-EXGTT,0.0,78.0,0.0,0.0,1.0,1.0,1.0,33.936291,-118.332639,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2995.070000
4,8350-NYMVI,0.0,80.0,0.0,0.0,1.0,1.0,1.0,36.192042,-118.020188,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3102.360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4628-WQCQQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,70.0,2944.270000
7039,2462-XIIJB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,343.450000
7040,5167-GBFRE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,0.0,3044.441077
7041,0388-EOPEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1942.950000


In [11]:
status['Churn Category'] = status['Churn Category'].replace(['No Churn','Competitor','Dissatisfaction','Attitude','Price','Other'],[0,1,2,3,4,5])
status['Churn Category'] = status['Churn Category'].fillna(method='pad')
status['Churn Category'] = status['Churn Category'].astype(int)

x = pd.merge(x,status,on='Customer ID',how='outer')



In [12]:
x = pd.merge(Train_ID,x,on='Customer ID',how='left')
#x.to_csv("csv_combine.csv",sep=',')

pd.DataFrame(x)

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,Latitude,Longitude,...,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Total Extra Data Charges,Total Revenue,Churn Category
0,0650-BWOZN,1.0,37.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3923.45,1.0
1,0562-FGDCR,0.0,54.0,0.0,0.0,0.0,0.0,0.0,34.903052,-118.411251,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,35.90,1.0
2,6688-UZPWD,1.0,46.0,1.0,1.0,0.0,1.0,0.0,33.721917,-118.043237,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2182.45,0.0
3,2905-KFQUV,1.0,23.0,1.0,1.0,0.0,0.0,0.0,36.192042,-122.000887,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,10.0,705.61,1.0
4,9720-JJJOR,0.0,41.0,0.0,0.0,1.0,0.0,0.0,39.672813,-120.456699,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,NaN,3075.66,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,1178-PZGAB,1.0,27.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,62.35,NaN
5630,4806-KEXQR,0.0,50.0,0.0,0.0,0.0,0.0,0.0,37.140104,-119.657092,...,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,3714.50,NaN
5631,8809-RIHDD,1.0,60.0,0.0,1.0,0.0,1.0,2.0,NaN,NaN,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1396.02,0.0
5632,6663-JOCQO,0.0,46.0,0.0,0.0,0.0,1.0,3.0,36.192042,-119.754941,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1699.88,0.0


In [13]:
x['Gender'] = x['Gender'].fillna(method='pad')
x['Age'] = x['Age'].fillna(method='pad')
x['Under_30'] = x['Under_30'].fillna(method='pad')
x['Senior_Citizen'] = x['Senior_Citizen'].fillna(method='pad')
x['Married'] = x['Married'].fillna(method='pad')
x['Dependents'] = x['Dependents'].fillna(method='pad')
x['Number_of_Dependents'] = x['Number_of_Dependents'].fillna(method='pad')
x['Latitude'] = x['Latitude'].fillna(method='pad')
x['Longitude'] = x['Longitude'].fillna(method='pad')
x['Satisfaction Score'] = x['Satisfaction Score'].fillna(method='pad')
x['Quarter'] = x['Quarter'].fillna(method='pad')
x['Referred a Friend'] = x['Referred a Friend'].fillna(method='pad')
x['Number of Referrals'] = x['Number of Referrals'].fillna(method='pad')
x['Tenure in Months'] = x['Tenure in Months'].fillna(method='pad')
x['Offer'] = x['Offer'].fillna(method='pad')
x['Phone Service'] = x['Phone Service'].fillna(method='pad')
x['Multiple Lines'] = x['Multiple Lines'].fillna(method='pad')
x['Internet Service'] = x['Internet Service'].fillna(method='pad')
x['Internet Type'] = x['Internet Type'].fillna(method='pad')
x['Avg Monthly GB Download'] = x['Avg Monthly GB Download'].fillna(method='pad')
x['Online Security'] = x['Online Security'].fillna(method='pad')
x['Online Backup'] = x['Online Backup'].fillna(method='pad')
x['Device Protection Plan'] = x['Device Protection Plan'].fillna(method='pad')
x['Premium Tech Support'] = x['Premium Tech Support'].fillna(method='pad')
x['Streaming Movies'] = x['Streaming Movies'].fillna(method='pad')
x['Streaming Music'] = x['Streaming Music'].fillna(method='pad')
x['Unlimited Data'] = x['Unlimited Data'].fillna(method='pad')
x['Contract'] = x['Contract'].fillna(method='pad')
x['Paperless Billing'] = x['Paperless Billing'].fillna(method='pad')
x['Payment Method'] = x['Payment Method'].fillna(method='pad')
x['Total Extra Data Charges'] = x['Total Extra Data Charges'].fillna(method='pad')
x['Total Revenue'] = x['Total Revenue'].fillna(method='pad')
x['Churn Category'] = x['Churn Category'].fillna(method='pad')

pd.DataFrame(x)

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,Latitude,Longitude,...,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Total Extra Data Charges,Total Revenue,Churn Category
0,0650-BWOZN,1.0,37.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3923.45,1.0
1,0562-FGDCR,0.0,54.0,0.0,0.0,0.0,0.0,0.0,34.903052,-118.411251,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,35.90,1.0
2,6688-UZPWD,1.0,46.0,1.0,1.0,0.0,1.0,0.0,33.721917,-118.043237,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2182.45,0.0
3,2905-KFQUV,1.0,23.0,1.0,1.0,0.0,0.0,0.0,36.192042,-122.000887,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,10.0,705.61,1.0
4,9720-JJJOR,0.0,41.0,0.0,0.0,1.0,0.0,0.0,39.672813,-120.456699,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,10.0,3075.66,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,1178-PZGAB,1.0,27.0,1.0,1.0,1.0,0.0,0.0,36.192042,-121.396719,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,62.35,0.0
5630,4806-KEXQR,0.0,50.0,0.0,0.0,0.0,0.0,0.0,37.140104,-119.657092,...,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,3714.50,0.0
5631,8809-RIHDD,1.0,60.0,0.0,1.0,0.0,1.0,2.0,37.140104,-119.657092,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1396.02,0.0
5632,6663-JOCQO,0.0,46.0,0.0,0.0,0.0,1.0,3.0,36.192042,-119.754941,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1699.88,0.0


In [14]:
#x.to_csv("csv_combine.csv",sep=',')

x['Latitude'] = x['Latitude'].fillna(method='bfill')
x['Longitude'] = x['Longitude'].fillna(method='bfill')
#x.to_csv("csv_combine.csv",sep=',')
x.isnull().sum()

Customer ID                   0
Gender                        0
Age                           0
Under_30                      0
Senior_Citizen                0
Married                       0
Dependents                    0
Number_of_Dependents          0
Latitude                      0
Longitude                     0
Satisfaction Score            0
Quarter                       0
Referred a Friend             0
Number of Referrals           0
Tenure in Months              0
Offer                         0
Phone Service                 0
Multiple Lines                0
Internet Service              0
Internet Type                 0
Avg Monthly GB Download       0
Online Security               0
Online Backup                 0
Device Protection Plan        0
Premium Tech Support          0
Streaming TV                698
Streaming Movies              0
Streaming Music               0
Unlimited Data                0
Contract                      0
Paperless Billing             0
Payment 

In [15]:
#seperate feature and label
x_training = x.iloc[:, :-1].values
y_training = x.iloc[:, -1].values

x_training

array([['0650-BWOZN', 1.0, 37.0, ..., 0.0, 0.0, 3923.45],
       ['0562-FGDCR', 0.0, 54.0, ..., 0.0, 0.0, 35.9],
       ['6688-UZPWD', 1.0, 46.0, ..., 0.0, 0.0, 2182.45],
       ...,
       ['8809-RIHDD', 1.0, 60.0, ..., 1.0, 0.0, 1396.02],
       ['6663-JOCQO', 0.0, 46.0, ..., 1.0, 0.0, 1699.88],
       ['7010-ZMVBF', 1.0, 71.0, ..., 0.0, 0.0, 899.24]], dtype=object)

In [180]:
x.to_csv('training.csv',sep=',')

In [16]:
pd.DataFrame(x_training)

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0650-BWOZN,1.0,37.0,0.0,1.0,0.0,0.0,0.0,34.903052,-118.411251,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,3923.45
1,0562-FGDCR,0.0,54.0,0.0,0.0,0.0,0.0,0.0,34.903052,-118.411251,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,35.9
2,6688-UZPWD,1.0,46.0,1.0,1.0,0.0,1.0,0.0,33.721917,-118.043237,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2182.45
3,2905-KFQUV,1.0,23.0,1.0,1.0,0.0,0.0,0.0,36.192042,-122.000887,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,10.0,705.61
4,9720-JJJOR,0.0,41.0,0.0,0.0,1.0,0.0,0.0,39.672813,-120.456699,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,10.0,3075.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,1178-PZGAB,1.0,27.0,1.0,1.0,1.0,0.0,0.0,36.192042,-121.396719,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,62.35
5630,4806-KEXQR,0.0,50.0,0.0,0.0,0.0,0.0,0.0,37.140104,-119.657092,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,3714.5
5631,8809-RIHDD,1.0,60.0,0.0,1.0,0.0,1.0,2.0,37.140104,-119.657092,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1396.02
5632,6663-JOCQO,0.0,46.0,0.0,0.0,0.0,1.0,3.0,36.192042,-119.754941,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1699.88


In [18]:
def signLogic(n) :
    if n > 0:
        return 1
    else :
        return -1

def sigmoid(s):
    return 1 / (1 + np.exp(-s))

def errorRate_Log01(w, x, y):
    y_Hat = np.array(list(map(signLogic,x.dot(w))))    
    return (np.sum(y != y_Hat) / (10*len(x)))

def LogicReg(Eta, w, x, y):
    for i in range(500):
        for j in range(4507) :
            #r = rd.randint(0, len(x)-1)
            t = sigmoid((-1.0)*y[j] * w.T.dot(x[j])) * y[j] * x[j]
        t = t*Eta
        w = w + t
    return w

In [19]:
Eta = 0.1
w0 = np.zeros(33)
y_NoChurn = []
y_Competitor = []
y_Dissatisfaction = []
y_Attitude = []
y_Price = []
y_Other = []

#split data to test and train from x_training
#modify triain_y to 5 category
#x_train,y_train : training data and label
#x_test,y_test : testind dataand label
#x_training = training data before splitting
#y_training = label of training data before splitting
#x : dataframe combining x and its' label 
x_train,x_test,y_train,y_test  = train_test_split(x_training,y_training,random_state=42,test_size=0.2)

x_train = x_train[:, 1:]
x_test = x_test[:, 1:]
for i in y_train:
    if i == 0 :
        y_NoChurn.append(1)
        y_Competitor.append(-1)
        y_Dissatisfaction.append(-1)
        y_Attitude.append(-1)
        y_Price.append(-1)
        y_Other.append(-1)
    elif i == 1 :
        y_NoChurn.append(-1)
        y_Competitor.append(1)
        y_Dissatisfaction.append(-1)
        y_Attitude.append(-1)
        y_Price.append(-1)
        y_Other.append(-1)
    elif i == 2 :
        y_NoChurn.append(-1)
        y_Competitor.append(-1)
        y_Dissatisfaction.append(1)
        y_Attitude.append(-1)
        y_Price.append(-1)
        y_Other.append(-1)
    elif i == 3 :
        y_NoChurn.append(-1)
        y_Competitor.append(-1)
        y_Dissatisfaction.append(-1)
        y_Attitude.append(1)
        y_Price.append(-1)
        y_Other.append(-1)
    elif i == 4 :
        y_NoChurn.append(-1)
        y_Competitor.append(-1)
        y_Dissatisfaction.append(-1)
        y_Attitude.append(-1)
        y_Price.append(1)
        y_Other.append(-1)
    elif i == 5 :
        y_NoChurn.append(-1)
        y_Competitor.append(-1)
        y_Dissatisfaction.append(-1)
        y_Attitude.append(-1)
        y_Price.append(-1)
        y_Other.append(1)


w_NC = LogicReg(Eta,w0,x_train,y_NoChurn)
w_Competitor = LogicReg(Eta,w0,x_train,y_Competitor)
w_Dissatisfaction = LogicReg(Eta,w0,x_train,y_Dissatisfaction)
w_Attitude = LogicReg(Eta,w0,x_train,y_Attitude)
w_Price = LogicReg(Eta,w0,x_train,y_Price)
w_Other = LogicReg(Eta,w0,x_train,y_Other)


/Users/johnny/Documents/MachineLearning/HW/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


In [21]:
#w_each^T * x_test = y_predict then pick the max one(probability),would be one of six w
#Next,comparing with y_test which is an answer then calulate the error rate is avilable

w_dict = {}
predict = []
dict = ['w_NC','w_Competitor','w_Dissatisfaction','w_Attitude','w_Price','w_Other']
w_dict['w_NC'] = w_NC
w_dict['w_Competitor'] = w_Competitor
w_dict['w_Dissatisfaction'] = w_Dissatisfaction
w_dict['w_Attitude'] = w_Attitude
w_dict['w_Price'] = w_Price
w_dict['w_Other'] = w_Other

max_score  = -1
sum = 0
for i in range(len(x_test)):
    for j in range(0,6):
        score = x_test[i].dot(w_dict[dict[j]])
        score = sigmoid(score)
        if score > max_score : 
            max_score = score
            label = i
    predict.append(label)
    print(predict[i],y_test[i])
    if predict[i] == y_test[i] :
        sum = sum + 1

acc = sum / len(y_test)
print(acc)

/Users/johnny/Documents/MachineLearning/HW/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


0 0.0
0 0.0
0 3.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 5.0
0 0.0
0 0.0
0 0.0
0 0.0
0 3.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 4.0
0 5.0
0 0.0
0 0.0
0 0.0
0 2.0
0 4.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 5.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 3.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 2.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 2.0
0 0.0
0 0.0
0 0.0
0 2.0
0 0.0
0 0.0
0 0.0
0 2.0
0 0.0
0 0.0
0 0.0
0 0.0
0 4.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 2.0
0 3.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 4.0
0 0.0
0 0.0
0 0.0
0 4.0
0 4.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 5.0
0 1.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 1.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 5.0
0 0.0
0 0.0
0 2.0
0 0.0
0 5.0
0 0.

In [1]:
#6 models in w_dict[],
# each will return a possibility for the class they represents to,
#input like below to have a predicition result and export to a csv file
#for i in range(len(x_test)):
    #for j in range(0,6):
        #score = x_test[i].dot(w_dict[dict[j]])
        #score = sigmoid(score)
        #if score > max_score : 
            #max_score = score
            #label = i
    #predict.append(label)

In [38]:
submission_combination_Logregression = pd.merge(Test_ID,demographics,on='Customer ID',how='left')
submission_combination_Logregression = pd.merge(submission_combination_Logregression,location,on='Customer ID',how='left')
submission_combination_Logregression = pd.merge(submission_combination_Logregression,satisfaction,on='Customer ID',how='left')
submission_combination_Logregression = pd.merge(submission_combination_Logregression,services,on='Customer ID',how='left')
submission_combination_Logregression = pd.merge(submission_combination_Logregression,status,on='Customer ID',how='left')

submission_combination_Logregression

,Customer ID,Count_x,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Count_y,...,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Total Extra Data Charges,Total Revenue,Churn Category
0,9938-EKRGF,1.0,Female,73.0,No,Yes,Yes,No,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4548.910000,NaN
1,7379-POKDZ,1.0,Male,64.0,No,No,No,No,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3553.250000,NaN
2,0654-HMSHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,4406.250000,NaN
3,2045-BMBTJ,1.0,Female,35.0,No,No,Yes,Yes,3.0,NaN,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,499.410000,NaN
4,0701-TJSEF,NaN,Male,67.0,No,Yes,Yes,No,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3044.441077,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4587-VVTOX,NaN,Male,49.0,No,No,No,No,NaN,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,387.700000,NaN
1405,7716-YTYHG,1.0,Female,78.0,No,Yes,No,No,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1406,7649-PHJVR,1.0,Male,27.0,Yes,No,Yes,No,0.0,NaN,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3044.441077,NaN
1407,7855-DIWPO,1.0,Female,38.0,NaN,NaN,NaN,NaN,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,640.780000,NaN


In [39]:
submission_combination_Logregression = submission_combination_Logregression.drop(labels=['Count_x','Count_y','Country','State','City','Zip Code','Lat Long','Churn Category'],axis=1)
submission_combination_Logregression

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Latitude,Longitude,...,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Total Extra Data Charges,Total Revenue
0,9938-EKRGF,Female,73.0,No,Yes,Yes,No,0.0,33.141265,-116.967221,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4548.910000
1,7379-POKDZ,Male,64.0,No,No,No,No,0.0,34.231318,-117.662032,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3553.250000
2,0654-HMSHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.802959,-117.027095,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,4406.250000
3,2045-BMBTJ,Female,35.0,No,No,Yes,Yes,3.0,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,499.410000
4,0701-TJSEF,Male,67.0,No,Yes,Yes,No,0.0,33.581045,-117.147190,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3044.441077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,4587-VVTOX,Male,49.0,No,No,No,No,NaN,37.871416,-119.754941,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,387.700000
1405,7716-YTYHG,Female,78.0,No,Yes,No,No,0.0,40.448632,-119.754941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1406,7649-PHJVR,Male,27.0,Yes,No,Yes,No,0.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,3044.441077
1407,7855-DIWPO,Female,38.0,NaN,NaN,NaN,NaN,3.0,33.688546,-119.754941,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,640.780000


In [41]:
submission_combination_Logregression['Gender']=submission_combination_Logregression['Gender'].replace(['Male','Female'],[1,0])
submission_combination_Logregression['Under 30']=submission_combination_Logregression['Under 30'].replace(['Yes','No'],[1,0])
submission_combination_Logregression['Senior Citizen']=submission_combination_Logregression['Senior Citizen'].replace(['Yes','No'],[1,0])
submission_combination_Logregression['Married']=submission_combination_Logregression['Married'].replace(['Yes','No'],[1,0])
submission_combination_Logregression['Dependents']=submission_combination_Logregression['Dependents'].replace(['Yes','No'],[1,0])

submission_combination_Logregression.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1409 entries, 0 to 1408
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               1409 non-null   object 
 1   Gender                    1085 non-null   float64
 2   Age                       1077 non-null   float64
 3   Under 30                  1056 non-null   float64
 4   Senior Citizen            1079 non-null   float64
 5   Married                   1076 non-null   float64
 6   Dependents                1088 non-null   float64
 7   Number of Dependents      1075 non-null   float64
 8   Latitude                  1215 non-null   float64
 9   Longitude                 1215 non-null   float64
 10  Satisfaction Score        1222 non-null   float64
 11  Quarter                   1227 non-null   float64
 12  Referred a Friend         1227 non-null   float64
 13  Number of Referrals       1227 non-null   float64
 14  Tenure i

In [43]:
submission_combination_Logregression.to_csv('SampleSubmission_withoutFillna.csv',sep=',')

In [52]:
x_df = pd.DataFrame(x_training)
print(x_df.isnull().sum())

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25    698
26      0
27      0
28      0
29      0
30      0
31      0
32      0
33      0
dtype: int64
